# Importando as bibliotecas





In [1]:
import tensorflow as tf
tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
tf.config.list_physical_devices('GPU')


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17896301344129938870
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2252026676
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10151996334010579877
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
gpus = tf.config.list_logical_devices('GPU')

type(gpus)
if gpus:
  # Replicate your computation on multiple GPUs
  c = []
  for gpu in gpus:
    with tf.device(gpu.name):
        print(gpu.name)
        pass
gpu = gpus[0]

/device:GPU:0


In [4]:
import sys
import os
from sklearn.model_selection import train_test_split
sys.path.append('./')
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import ThresholdedReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import models
import h5py
from tensorflow.keras.models import Model
from keras import Input
from tensorflow.keras import optimizers
import json
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import sklearn.metrics as sklm
import pandas as pd

In [5]:
CHARNN_DATA_CONFIG = "D:/SiDi/Project/Modulo II/github/multimodal-sentiment-analysis/rbsa2/charnn_cnn/charcnn_data_config.json"
charcnn_data_config = pd.read_json(CHARNN_DATA_CONFIG)

CNN_DATA_CONFIG = "D:/SiDi/Project/Modulo II/github/multimodal-sentiment-analysis/rbsa2/charnn_cnn/nn_config.json"
nn_data_config = pd.read_json(CNN_DATA_CONFIG)


In [6]:
#data_config = json.load(open("charcnn_data_config.json")) #Leitura do arquivo de configuracao
data_config = charcnn_data_config #Leitura do arquivo de configuracao

#nn_config = json.load(open("nn_config.json")) # Leitura de arquivo de configuracao, quais sao os parametros
nn_config = nn_data_config # Leitura de arquivo de configuracao, quais sao os parametros

alphabet_size = data_config["data"]["alphabet_size"]+1 #tamanho do alfabeto
input_size = data_config["data"]["input_size"] #Tamanho do input de zi

fully_connected_layers = nn_config["char_cnn_zhang"]["fully_connected_layers"] #Camadas Conectadas
conv_layers = nn_config["char_cnn_zhang"]["conv_layers"] #Camadas Convulacionasi
threshold = nn_config["char_cnn_zhang"]["threshold"] # Limiar
dropout_p = nn_config["char_cnn_zhang"]["dropout_p"] #Taxa de Dropout
embedding_size = nn_config["char_cnn_zhang"]["embedding_size"] # Tamnho do Embedding

In [7]:
DATASET_PATH_URL  = "D:/SiDi/Project/Modulo II/dataset/output/dataset_sidi_v1.2.csv"
df = pd.read_csv(DATASET_PATH_URL, sep='\t')

print(len(df))
df.reset_index(inplace=True)

470586


In [8]:
df.head()

,index,Unnamed: 0,image_name,NEG,NEU,POS,text,sent_text,sent_image,image_path,image_height,image_width
0,0,0,768097808037605376-1,0.919993,0.055681,0.024326,"RT @hoseokahhh: ""who is this song forwhat do i...",NEG,0,data/76809/768097808037605376-1.jpg,449,449
1,1,1,768097833215938560-1,0.864726,0.058440,0.076834,RT @ARapperSaid_: “I ain’t come over here for ...,NEG,0,data/76809/768097833215938560-1.jpg,398,593
2,2,2,768097929659817984-1,0.907574,0.061396,0.031029,RT @blackedfriction: i hate telling people i l...,NEG,0,data/76809/768097929659817984-1.jpg,489,1132
3,3,3,768097929659817984-2,0.907574,0.061396,0.031029,RT @blackedfriction: i hate telling people i l...,NEG,0,data/76809/768097929659817984-2.jpg,983,1123
4,4,4,768097929659817984-3,0.907574,0.061396,0.031029,RT @blackedfriction: i hate telling people i l...,NEG,0,data/76809/768097929659817984-3.jpg,1326,1110


In [9]:
# Verifica o balanceamento das classes
classes = df.groupby('sent_text')['index'].nunique()/df.shape[0] #Agrupe por index
classes

sent_text
NEG    0.333333
NEU    0.333333
POS    0.333333
Name: index, dtype: float64

In [10]:
image_height = 224 #Defina o tamanho  da imagem
image_width = 224 #Defina a largura da iamge
from tensorflow.keras.preprocessing.image import load_img #carrega funcao para carraegar imagem
from tensorflow.keras.preprocessing.image import img_to_array #carrega funcao para converter imagem para array
from PIL import Image
#Funcao que remove quebra de linha
def remove_spaces(cell):  
    return cell.replace('\n',' ')
#Converte que carrega a image e depois convertte para array
def get_array(image_path):
    img = load_img("D:/SiDi/Project/Modulo II/dataset/b-t4sa_imgs" + '/' + image_path, target_size=(image_height, image_width), color_mode='grayscale')

    return img_to_array(img)


#Mapeaa os tipos de classes existente nos rotulos de predicoes
def to_one_hot(y, class_mapping, num_classes):
    y_int = y.map(class_mapping)
    return to_categorical(y_int, num_classes)

#X_text = df['text'].apply(remove_spaces)


# Numero de classes do dataset

In [11]:

num_classes = df['sent_text'].nunique()
#Faz o mapeeamento da classe com o id
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['sent_text']))}
class_mapping
#y_int = df['sent_text'].map(class_mapping)

{'NEG': 0, 'NEU': 1, 'POS': 2}

In [12]:
df.head()

,index,Unnamed: 0,image_name,NEG,NEU,POS,text,sent_text,sent_image,image_path,image_height,image_width
0,0,0,768097808037605376-1,0.919993,0.055681,0.024326,"RT @hoseokahhh: ""who is this song forwhat do i...",NEG,0,data/76809/768097808037605376-1.jpg,449,449
1,1,1,768097833215938560-1,0.864726,0.058440,0.076834,RT @ARapperSaid_: “I ain’t come over here for ...,NEG,0,data/76809/768097833215938560-1.jpg,398,593
2,2,2,768097929659817984-1,0.907574,0.061396,0.031029,RT @blackedfriction: i hate telling people i l...,NEG,0,data/76809/768097929659817984-1.jpg,489,1132
3,3,3,768097929659817984-2,0.907574,0.061396,0.031029,RT @blackedfriction: i hate telling people i l...,NEG,0,data/76809/768097929659817984-2.jpg,983,1123
4,4,4,768097929659817984-3,0.907574,0.061396,0.031029,RT @blackedfriction: i hate telling people i l...,NEG,0,data/76809/768097929659817984-3.jpg,1326,1110


In [13]:
"""docs = {}
#Identificador da imagem para agrupamento
values = df['image_name'].value_counts()[0:100]
for i in values.index:
    docs[i] = df[df['image_name'] == i].sort_values(by=['image_name'])#adicionar numa lista de documentos no chave valor

"""

"docs = {}\n#Identificador da imagem para agrupamento\nvalues = df['image_name'].value_counts()[0:100]\nfor i in values.index:\n    docs[i] = df[df['image_name'] == i].sort_values(by=['image_name'])#adicionar numa lista de documentos no chave valor\n\n"

In [14]:
"""len(df['image_name'][0])
df['image_name'][0]"""

"len(df['image_name'][0])\ndf['image_name'][0]"

In [15]:
"""#Iterar sobre cada imagem representad apenas com a chave
X_keys = [key for key in docs.keys()]
#Divide em teste e e validação
train_size = int(len(X_keys)*0.70)
print(len(X_keys[:train_size]))
print(len(X_keys[train_size:]))"""

'#Iterar sobre cada imagem representad apenas com a chave\nX_keys = [key for key in docs.keys()]\n#Divide em teste e e validação\ntrain_size = int(len(X_keys)*0.70)\nprint(len(X_keys[:train_size]))\nprint(len(X_keys[train_size:]))'

In [16]:
X_train_text = []
X_train_image = []
y_train = []
image_path = []
y_total = []
img_total = []
text_total = []
page_total = []
doc_total = []
target_total = []

for i in range(1,len(df[:71]),1):
    X_image = df[i-1:i]['image_path'].apply(get_array)
    X_text = df[i-1:i]['text']
    y = to_one_hot(df[ i-1:i]['sent_text'], class_mapping, num_classes)
    
    text_image = pd.concat([X_image, X_text], axis=1)
    
    image_path.extend( df[i-1:i]['image_path'])
    X_train_text.extend(X_text)
    X_train_image.extend(X_image)
    y_train.extend(y)
    y_total.append(y)
    text_total.append(X_text)
    img_total.append(X_image)
    page_total.append( df[i-1:i]['image_name'])
    doc_total.append( df[i-1:i]['image_name'])
    target_total.append( df[i-1:i]['sent_text'])
    
    
    
X_train_text = pd.Series(X_train_text)
X_train_image = pd.Series(X_train_image)
y_train = np.array(y_train)


In [17]:
len(y_total)
len(text_total)
len(img_total)

70

In [18]:
X_test_text = []
X_test_image = []
y_test = []

image_path = []
y_total = []
img_total = []
text_total = []
page_total = []
doc_total = []
target_total = []

for i in range(1,len(df[70:101]),1):
    X_image = df[i-1:i]['image_path'].apply(get_array)
    X_text = df[i-1:i]['text']
    y = to_one_hot(df[i-1:i]['sent_text'], class_mapping, num_classes)
    
    text_image = pd.concat([X_image, X_text], axis=1)
    
    X_test_text.extend(X_text)
    X_test_image.extend(X_image)
    y_test.extend(y)
    y_total.append(y)
    text_total.append(X_text)
    img_total.append(X_image)
    page_total.append(df[i-1:i]['image_name'])
    doc_total.append(df[i-1:i]['image_name'])
    target_total.append(df[i-1:i]['sent_text'])
    
    
X_test_text = pd.Series(X_test_text)
X_test_image = pd.Series(X_test_image)
y_test = np.array(y_test)

In [19]:
len(y_total), len(text_total), len(img_total)

(30, 30, 30)

In [20]:
class_mapping

{'NEG': 0, 'NEU': 1, 'POS': 2}

# Join Train and Test (Test and Image)

In [21]:
X_join_text = X_train_text.append(X_test_text)
X_join_image = X_train_image.append(X_test_image)
len(X_join_text)

100

# Carrega Dados para Validacao ou Teste

# Carrega objeto de Dados para servir de entrada na rede neural

In [22]:
class Data(object):
    """
    Class to handle loading and processing of raw datasets.
    """
    def __init__(self, data_source,
                 alphabet=" abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}",
                 input_size=1014, num_of_classes=3, alphabet_size=71):
        """
        Initialization of a Data object.

        Args:
            data_source (str): Raw data file path
            alphabet (str): Alphabet of characters to index
            input_size (int): Size of input features
            num_of_classes (int): Number of classes in data
        """
        self.alphabet = alphabet
        self.alphabet_size = len(self.alphabet)
        self.dict = {}  # Maps each character to an integer
        self.no_of_classes = num_of_classes
        for idx, char in enumerate(self.alphabet):
            self.dict[char] = idx + 1
        self.length = input_size
        self.data_source = data_source

    def load_data(self):
        """
        Load raw data from the source file into data variable.

        Returns: None

        """
        data = []
        for doc in self.data_source:
            data.append(doc)
        self.data = np.array(data)

    def get_all_data(self):
        """
        Return all loaded data from data variable.

        Returns:
            (np.ndarray) Data transformed from raw to indexed form with associated one-hot label.

        """
        data_size = len(self.data)
        start_index = 0
        end_index = data_size
        batch_texts = self.data[start_index:end_index]
        batch_indices = []
        one_hot = np.eye(self.no_of_classes, dtype='int64')
        classes = []
        for c, s in batch_texts:
            batch_indices.append(self.str_to_indexes(s))
            c = int(c) - 1
            classes.append(one_hot[c])
        return np.asarray(batch_indices, dtype='int64'), np.asarray(classes)

    def str_to_indexes(self, s):
        """
        Convert a string to character indexes based on character dictionary.
        
        Args:
            s (str): String to be converted to indexes

        Returns:
            str2idx (np.ndarray): Indexes of characters in s

        """
        s = s.lower()
        max_length = min(len(s), self.length)
        str2idx = np.zeros(self.length, dtype='int64')
        for i in range(1, max_length + 1):
            c = s[-i]
            if c in self.dict:
                str2idx[i - 1] = self.dict[c]
        return str2idx

    def vectorize_sentences(self, data, char_indices):
        X = []

        for doc in data:
        
            x = [char_indices[w] if w in self.alphabet else 0 for w in doc.lower()]
            x2 = np.eye(self.alphabet_size + 1)[x]
            X.append(x2)
            
        return pad_sequences(X, maxlen=self.length, padding="pre")

    def get_target_data(self):
        """
        Return all loaded data from data variable.

        Returns:
            (np.ndarray) Data transformed from raw to indexed form with associated one-hot label.

        """
        data_size = len(self.data)
        start_index = 0
        end_index = data_size
        batch_texts = self.data[start_index:end_index]
        batch_indices = []
        one_hot = np.eye(self.no_of_classes, dtype='int64')
        classes = []
        for c, s in batch_texts:
            c = int(c)
            classes.append(one_hot[c])
        return np.asarray(classes)

    def get_data_3dim(self):
        '''
        transform training input to shape Number_of_Rows X MaxLen X Alphabet_size
        this is need to be done to avoid the use of embedding layer, wich is not supported on the Tflite converter
        '''
        char_indices = dict((c, i) for i, c in enumerate(self.alphabet,1))
        indices_char = dict((i, c) for i, c in enumerate(self.alphabet,1))

        
        train_data = self.vectorize_sentences(self.data, char_indices)
        train_data.shape
        return train_data


    

# Instancia os Objetos (Data) Treinamento e Validacao a partir do  conjunto de dados Treinamento e Validacao

In [23]:
training_data = Data(data_source=X_train_text,
                         alphabet=data_config["data"]["alphabet"],
                         input_size=data_config["data"]["input_size"],
                         num_of_classes=data_config["data"]["num_of_classes"],
                         alphabet_size=data_config["data"]["alphabet_size"])
training_data.load_data()

validation_data = Data(data_source=X_test_text,
                         alphabet=data_config["data"]["alphabet"],
                         input_size=data_config["data"]["input_size"],
                         num_of_classes=data_config["data"]["num_of_classes"],
                         alphabet_size=data_config["data"]["alphabet_size"])
validation_data.load_data()
    

X_train_text = training_data.get_data_3dim()
X_test_text = validation_data.get_data_3dim()

# Converte de Imagem para Array Numpy - image_to_numpy(image_df)

In [24]:
def image_to_numpy(image_df):
    all_images = []
    for i in range(len(image_df)):
        all_images.append(image_df.iloc[i])  
    return np.array(all_images)

X_train_image = image_to_numpy(X_train_image)
X_test_image = image_to_numpy(X_test_image)

In [25]:
merge_data = Data(data_source=X_join_text,
                         alphabet=data_config["data"]["alphabet"],
                         input_size=data_config["data"]["input_size"],
                         num_of_classes=data_config["data"]["num_of_classes"],
                         alphabet_size=data_config["data"]["alphabet_size"])
merge_data.load_data()
    

X_join_text = merge_data.get_data_3dim()

X_join_image = image_to_numpy(X_join_image)

# Imprime o Shape do Objeto de Treinamento Convertido Array Numpy 

In [26]:
print(X_train_image.shape)

(70, 224, 224, 1)


# (01) - Conjunto de Dados de Validacao - Declara a Classe ImageValidationCheckpoint

In [ ]:
def predict(model, X_test_image, batch_size=32):
    y_predict = np.round(model.predict(X_test_image))
    return y_predict

def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

class ImageValidationCheckpoint(Callback):
    def __init__(self, filepath, X_test_image,y_test, metric = 'kappa'):
        self.X_test_image = X_test_image
        self.y_test = y_test
        self.metric = metric
        self.max_metric = float('-inf')
        self.max_metrics = None
        self.filepath = filepath
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        
        predicted_labels = predict(self.model, X_test_image)
        #true_labels = [label2Idx[x[1]] for x in self.test_data]
        true_labels = one_hot_decode(self.y_test)
        predicted_labels = one_hot_decode(predicted_labels)
        eval_metrics = {
            'accuracy' : sklm.accuracy_score(true_labels, predicted_labels),
            'f1_micro' : sklm.f1_score(true_labels, predicted_labels, average='micro'),
            'f1_macro' : sklm.f1_score(true_labels, predicted_labels, average='macro'),
            'f1_binary' : sklm.f1_score(true_labels, predicted_labels, average='weighted', pos_label = 1),
            'kappa' : sklm.cohen_kappa_score(true_labels, predicted_labels)
        }
        eval_metric = eval_metrics[self.metric]
        self.history.append(eval_metric)
        
        if epoch > -1 and eval_metric > self.max_metric:
            print("\n" + self.metric + " improvement: " + str(eval_metric) + " (before: " + str(self.max_metric) + "), saving to " + self.filepath)
            self.max_metric = eval_metric     # optimization target
            self.max_metrics = eval_metrics   # all metrics
            #self.model.save(self.filepath)

image_input = Input(shape=(image_height, image_width, 1), name='image_input', dtype='float32')
x = layers.Conv2D(32, (8, 8), activation='tanh',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(32, (6, 6), activation='tanh')(x)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(32, (8, 8), activation='tanh',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(32, (6, 6), activation='tanh')(x)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Flatten()(x)
encoded_image = layers.Dense(256, activation='tanh', name='encoded_image')(x)

output = layers.Dense(num_classes, activation='softmax', name='output')(encoded_image)
model_image = Model(image_input, output)
model_image.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-4),
                    metrics=['acc'])
print(model_image.summary())

n_repeats = 1
exp_history = []
optimize_for = 'kappa'
model_file = 'model_ness_txt_image_v1_test.h5'
with tf.device(gpu.name):
    for i in range(n_repeats):
        checkpoint = ImageValidationCheckpoint(model_file, X_test_image, y_test, metric=optimize_for)
        model_image.fit(X_train_image, y_train, epochs=40, batch_size=32,
                         validation_data=(X_test_image, y_test)
                         ,callbacks = [checkpoint])
        exp_history.append(checkpoint.max_metrics)
    
avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp_history]) / n_repeats
print(avg_result)
print(avg_acc)
for i, r in enumerate(exp_history):
    model_file = "ness_exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)
model_image.save('model_ness_txt_image_v1.h5')

D:\SiDi\Project\Modulo II\github\keras-tf-gpu\keras-tf-gpu\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image_input (InputLayer)    [(None, 224, 224, 1)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 217, 217, 32)      2080      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 31, 31, 32)        36896     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 800)               0     

# (01) - Configuracao da Arquitetura I 

1.   Conv2D -> Inicial(32 (8,8)) e Oculta (32 (6,6))
2.   MaxPooling -> (32 (6,6))
3.   Loss -> categorical_crossentropy   
4.   optimizer -> optimizers.RMSprop(lr=1e-4)




In [ ]:
image_input = Input(shape=(image_height, image_width, 1), name='image_input', dtype='float32')
x = layers.Conv2D(32, (8, 8), activation='tanh',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(32, (6, 6), activation='tanh')(x)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(32, (8, 8), activation='tanh',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(32, (6, 6), activation='tanh')(x)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Flatten()(x)
encoded_image = layers.Dense(256, activation='tanh', name='encoded_image')(x)

output = layers.Dense(num_classes, activation='softmax', name='output')(encoded_image)
model_image = Model(image_input, output)
model_image.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(learning_rate=1e-4),
                    metrics=['acc'])
print(model_image.summary())



# (01) - Avaliacao da Arquitetura I

In [ ]:
n_repeats = 1
exp_history = []
optimize_for = 'kappa'
model_file = 'model_ness_txt_image_v1_test.h5'
for i in range(n_repeats):
    checkpoint = ImageValidationCheckpoint(model_file, X_test_image, y_test, metric=optimize_for)
    model_image.fit(X_train_image, y_train, epochs=40, batch_size=32,
                     validation_data=(X_test_image, y_test)
                     ,callbacks = [checkpoint])
    exp_history.append(checkpoint.max_metrics)
    
avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp_history]) / n_repeats
print(avg_result)
print(avg_acc)
for i, r in enumerate(exp_history):
    model_file = "ness_exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)
model_image.save('model_ness_txt_image_v1.h5')

# (02) - Conjunto de Dados de Validacao Declara a Classe ImageValidationCheckpoint 



In [ ]:
def predict(model, X_test_image, batch_size=32):
    y_predict = np.round(model.predict(X_test_image))
    return y_predict

def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

class ImageValidationCheckpoint(Callback):
    def __init__(self, filepath, X_test_image,y_test, metric = 'kappa'):
        self.X_test_image = X_test_image
        self.y_test = y_test
        self.metric = metric
        self.max_metric = float('-inf')
        self.max_metrics = None
        self.filepath = filepath
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        
        predicted_labels = predict(self.model, X_test_image)
        #true_labels = [label2Idx[x[1]] for x in self.test_data]
        true_labels = one_hot_decode(self.y_test)
        predicted_labels = one_hot_decode(predicted_labels)
        eval_metrics = {
            'accuracy' : sklm.accuracy_score(true_labels, predicted_labels),
            'f1_micro' : sklm.f1_score(true_labels, predicted_labels, average='micro'),
            'f1_macro' : sklm.f1_score(true_labels, predicted_labels, average='macro'),
            #'f1_binary' : sklm.f1_score(true_labels, predicted_labels, average='weighted', pos_label = 1),
            'kappa' : sklm.cohen_kappa_score(true_labels, predicted_labels)
        }
        eval_metric = eval_metrics[self.metric]
        self.history.append(eval_metrics)
        
        if epoch > -1 and eval_metric > self.max_metric:
            print("\n" + self.metric + " improvement: " + str(eval_metric) + " (before: " + str(self.max_metric) + "), saving to " + self.filepath)
            self.max_metric = eval_metric     # optimization target
            self.max_metrics = eval_metrics   # all metrics
            #self.model.save(self.filepath)




# (02) - Configuracao da Arquitetura II


1.   Conv2D -> Inicial(16,(8,8)) e Oculta (16,(6, 6))
2.   MaxPooling -> (6,6)
3.   Loss -> categorical_crossentropy   
4.   optimizer -> optimizers.RMSprop(lr=1e-4)

In [ ]:
image_input = Input(shape=(image_height, image_width, 1), name='image_input', dtype='float32')
x = layers.Conv2D(16, (8, 8), activation='tanh',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(16, (6, 6), activation='tanh')(x)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(16, (8, 8), activation='tanh',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(16, (6, 6), activation='tanh')(x)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Flatten()(x)
encoded_image = layers.Dense(256, activation='tanh', name='encoded_image')(x)

output = layers.Dense(num_classes, activation='softmax', name='output')(encoded_image)
model_image = Model(image_input, output)
model_image.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-4),
                    metrics=['acc'])
print(model_image.summary())


# (02) - Avaliacao da Arquitetura II

In [ ]:
n_repeats = 1
exp_history = []
optimize_for = 'kappa'
model_file = 'model_ness_txt_image_v1_test.h5'
for i in range(n_repeats):
    checkpoint = ImageValidationCheckpoint(model_file, X_test_image, y_test, metric=optimize_for)
    model_image.fit(X_train_image, y_train, epochs=40, batch_size=16,
                     validation_data=(X_test_image, y_test)
                     ,callbacks = [checkpoint])
    exp_history.append(checkpoint.max_metrics)
    
avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp_history]) / n_repeats
print(avg_result)
print(avg_acc)
for i, r in enumerate(exp_history):
    model_file = "ness_exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)
model_image.save('model_ness_txt_image_v1.h5')

# Teste o modelo treinado

In [ ]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
import time
start = time.time()
y_pred = model_image.predict(X_test_image)
print(time.time()-start)

# Exporte o modelo 

In [ ]:
import pickle
pickle.dump( checkpoint.history, open( "image-model-aibox40.pickle", "wb" ) )
print(checkpoint.history)

In [ ]:
print(avg_result)
print(avg_acc)

# (03) - Conjunto de Dados de Validacao Declara a Classe ImageValidationCheckpoint 



In [ ]:
def predict(model, X_test_text, batch_size=32):
    y_predict = np.round(model.predict(X_test_text))
    return y_predict

def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

class TextValidationCheckpoint(Callback):
    def __init__(self, filepath, X_test_text,y_test, metric = 'kappa'):
        self.X_test_text = X_test_text
        self.y_test = y_test
        self.metric = metric
        self.max_metric = float('-inf')
        self.max_metrics = None
        self.filepath = filepath
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        
        predicted_labels = predict(self.model, self.X_test_text)
        true_labels = one_hot_decode(self.y_test)
        predicted_labels = one_hot_decode(predicted_labels)
        eval_metrics = {
            'accuracy' : sklm.accuracy_score(true_labels, predicted_labels),
            'f1_micro' : sklm.f1_score(true_labels, predicted_labels, average='micro'),
            'f1_macro' : sklm.f1_score(true_labels, predicted_labels, average='macro'),
            #'f1_binary' : sklm.f1_score(true_labels, predicted_labels, average='weighted', pos_label = 1),
            'kappa' : sklm.cohen_kappa_score(true_labels, predicted_labels)
        }
        eval_metric = eval_metrics[self.metric]
        self.history.append(eval_metrics)
        
        if epoch > -1 and eval_metric > self.max_metric:
            print("\n" + self.metric + " improvement: " + str(eval_metric) + " (before: " + str(self.max_metric) + "), saving to " + self.filepath)
            self.max_metric = eval_metric     # optimization target
            self.max_metrics = eval_metrics   # all metrics
            #self.model.save(self.filepath)



# (03) - Configuracao da Arquitetura II


1.   Conv2D -> Inicial(16,(8,8)) e Oculta (16,(6, 6))
2.   MaxPooling -> (6,6)
3.   Loss -> categorical_crossentropy   
4.   optimizer -> optimizers.RMSprop(lr=1e-4)

In [ ]:
text_input = Input(shape=(input_size, alphabet_size), name='text_input', dtype='float32')

x = Convolution1D(nb_filter=50, filter_length=3,
                     border_mode='valid', activation='tanh',
                     input_shape=(input_size, alphabet_size))(text_input)

for cl in conv_layers:
    x = Convolution1D(cl[0], cl[1])(x)
    #x = ThresholdedReLU(threshold)(x)
    if cl[2] != -1:
        x = MaxPooling1D(cl[2])(x)
x = Flatten()(x)

for fl in fully_connected_layers:
    x = Dense(fl)(x)
    #x = ThresholdedReLU(threshold)(x)
    x = Dropout(dropout_p)(x)

encoded_text = Dense(256, activation='tanh', name='encoded_text')(x)

output = layers.Dense(num_classes, activation='softmax', name='output')(encoded_text)
model_text = Model(text_input, output)
model_text.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-4),
                    metrics=['acc'])
print(model_text.summary())

# (03) - Treino da Rede Neural da Arquitetura III

In [ ]:
n_repeats = 1
exp_history = []
optimize_for = 'kappa'
model_file = 'model_ness_txt_image_v1_test.h5'
for i in range(n_repeats):
    checkpoint = TextValidationCheckpoint(model_file, X_test_text, y_test, metric=optimize_for)
    model_text.fit(X_train_text, y_train, epochs=40, batch_size=16,
                     validation_data=(X_test_text, y_test)
                     ,callbacks = [checkpoint])
    exp_history.append(checkpoint.max_metrics)
    
avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp_history]) / n_repeats
print(avg_result)
print(avg_acc)
for i, r in enumerate(exp_history):
    model_file = "ness_exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)
model_text.save('model_ness_txt_image_v1.h5')

model_text.save('model_ness_txt_v1.h5')

# (03) Avaliação da Rede Neural III

In [ ]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
import time
start = time.time()
y_pred = model_text.predict(X_test_text)
print(time.time()-start)

# (03) Exportação do Modelo da Rede Neural

In [ ]:
import pickle
pickle.dump( checkpoint.history, open( "text-model-aibox40.pickle", "wb" ) )
print(checkpoint.history)

# (04) - Conjunto de Dados de Validacao Declara a Classe ImageValidationCheckpoint 


In [ ]:
def predict(model, X_test_text, X_test_image, batch_size=32):
    y_predict = np.round(model.predict([X_test_text, X_test_image]))
    return y_predict

def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

class ValidationCheckpoint(Callback):
    def __init__(self, filepath, X_test_text, X_test_image,y_test, metric = 'kappa'):
        self.X_test_text = X_test_text
        self.X_test_image = X_test_image
        self.y_test = y_test
        self.metric = metric
        self.max_metric = float('-inf')
        self.max_metrics = None
        self.filepath = filepath
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        
        predicted_labels = predict(self.model, self.X_test_text, X_test_image)
        #true_labels = [label2Idx[x[1]] for x in self.test_data]
        true_labels = one_hot_decode(self.y_test)
        predicted_labels = one_hot_decode(predicted_labels)
        eval_metrics = {
            'accuracy' : sklm.accuracy_score(true_labels, predicted_labels),
            'f1_micro' : sklm.f1_score(true_labels, predicted_labels, average='micro'),
            'f1_macro' : sklm.f1_score(true_labels, predicted_labels, average='macro'),
            #'f1_binary' : sklm.f1_score(true_labels, predicted_labels, average='weighted', pos_label = 1),
            'kappa' : sklm.cohen_kappa_score(true_labels, predicted_labels)
        }
        eval_metric = eval_metrics[self.metric]
        self.history.append(eval_metrics)
        
        if epoch > -1 and eval_metric > self.max_metric:
            print("\n" + self.metric + " improvement: " + str(eval_metric) + " (before: " + str(self.max_metric) + "), saving to " + self.filepath)
            self.max_metric = eval_metric     # optimization target
            self.max_metrics = eval_metrics   # all metrics
            #self.model.save(self.filepath)

# (04) Configuração da Arquitetura (Imagem + Texto)

In [ ]:
image_input = Input(shape=(image_height, image_width, 1), name='image_input', dtype='float32')
x = layers.Conv2D(16, (8, 8), activation='tanh',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(16, (6, 6), activation='tanh')(x)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(16, (8, 8), activation='tanh',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(16, (6, 6), activation='tanh')(x)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Flatten()(x)
encoded_image = layers.Dense(256, activation='tanh', name='encoded_image')(x)



text_input = Input(shape=(input_size, alphabet_size), name='text_input', dtype='float32')

x = Convolution1D(nb_filter=50, filter_length=3,
                     border_mode='valid', activation='tanh',
                     input_shape=(input_size, alphabet_size))(text_input)

for cl in conv_layers:
    x = Convolution1D(cl[0], cl[1])(x)
    #x = ThresholdedReLU(threshold)(x)
    if cl[2] != -1:
        x = MaxPooling1D(cl[2])(x)
x = Flatten()(x)

for fl in fully_connected_layers:
    x = Dense(fl)(x)
    #x = ThresholdedReLU(threshold)(x)
    x = Dropout(dropout_p)(x)

encoded_text = Dense(256, activation='tanh', name='encoded_text')(x)

concatenated = layers.concatenate([encoded_text, encoded_image], axis=-1, name='concatanated')
output = layers.Dense(num_classes, activation='softmax', name='output')(concatenated)
model_hybrid = Model([text_input, image_input], output)
model_hybrid.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-4),
                    metrics=['acc'])
model_hybrid.summary()

# (04) - Treinamento da Rede Neural = Texto e Imagem

In [ ]:
n_repeats = 1
exp_history = []
optimize_for = 'kappa'
model_file = 'model_ness_txt_image_v1_test.h5'
for i in range(n_repeats):
    checkpoint = ValidationCheckpoint(model_file, X_test_text, X_test_image, y_test, metric=optimize_for)
    model_hybrid.fit([X_train_text, X_train_image], y_train, epochs=40, batch_size=32,
                     validation_data=([X_test_text, X_test_image], y_test)
                     ,callbacks = [checkpoint])
    exp_history.append(checkpoint.max_metrics)
    
avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp_history]) / n_repeats
for i, r in enumerate(exp_history):
    model_file = "ness_exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)
model_hybrid.save('model_ness_txt_image_v1.h5')

# (04)  Exportação Modelo da Rede Neural ( Texto & Imagem )

In [ ]:
import pickle
pickle.dump( checkpoint.history, open( "text-image-model-aibox40.pickle", "wb" ) )
print(checkpoint.history)

# 04 Single Page Repeat

In [ ]:
avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp_history]) / n_repeats
print(avg_result)
print(avg_acc)
for i, r in enumerate(exp_history):
    model_file = "ness_exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)

#(04) Recarrega Modelo Hibrido

In [ ]:
from tensorflow.keras.models import load_model
model_hybrid = load_model('model_ness_txt_image_v1.h5')

# (04) Avalia modelo Hibrido (Texto & Image)

In [ ]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
import time
start = time.time()
y_pred = model_hybrid.predict([X_test_text, X_test_image])
print(time.time()-start)

In [ ]:
#X_join_text = X_train_text.append(X_test_text)
#X_image = X_train_image.append(X_test_image)


print("done")

# Fase de Teste

In [ ]:
import time
start = time.time()
y_pred = model_hybrid.predict([X_join_text, X_join_image])

print(time.time()-start)

# Mapping Id of Class

In [ ]:
id_class_mapping = {idx:label for idx,label in enumerate(np.unique(df['sent_text']))}
id_class_mapping

In [ ]:
pred_class = []
for i in range(len(y_pred)):
  pred = np.where(y_pred[i] == np.amax(y_pred[i]))
  pred_class.append(id_class_mapping[pred[0][0]])

# To CSV

In [ ]:
df_seq = pd.DataFrame({'image_name': doc_order, 'image_page': page_order, 'sent_text': target_order, 'image_predict': pred_class})

df_seq.to_csv('sequence-classifier-output.csv')

In [ ]:
df_seq['match'] = np.where(df_seq['sent_text']==df_seq['image_predict'], 
                                           'yes', 'no')

In [ ]:
df_seq['match'].value_counts()


In [ ]:
df_seq.head(n=2970)[-11:]

In [ ]:
df_seq['match'].apply(lambda x: x == 'no')

In [ ]:
df_seq[df_seq['match'].apply(lambda x: x == 'no')]

# Mapping Real Class

In [ ]:
y_real = np.array(real_mapping_class)

real_mapping_class_seq = []
for i in range(len(y_real)):
  pred = np.where(y_real[i] == np.amax(y_real[i]))
  real_mapping_class_seq.append(pred[0][0])
  #print(pred[0][0])

# Mapping Pred Class

In [ ]:
pred_mapping_class = []
for i in range(len(y_pred)):
  pred = np.where(y_pred[i] == np.amax(y_pred[i]))
  pred_mapping_class.append(pred[0][0])

# Export Class

In [ ]:
df['real_map_class'] = real_mapping_class_seq
df['pred_map_class'] = pred_mapping_class

df.head()

df.to_csv('ness-seq.csv')

#04 Realiza o mapeamento da classe

In [ ]:
num_classes = df['sent_text'].nunique()
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['sent_text']))}
y_int = df['sent_text'].map(class_mapping)
print(class_mapping)

# Verifica a taxa de acertos com classe predita e classe esperada

In [ ]:
y_f = []
for i in range(len(y_pred)):
    pred = np.where(y_pred[i] == np.amax(y_pred[i]))
    y_f.append(pred[0][0])

print(classification_report(Y_test, y_f))

In [ ]:
{'anexo': 0, 'anotacao': 1, 'autos': 2, 'branco': 3, 'certidao': 4, 
 'citacao': 5, 'decisao': 6, 'intimacao': 7, 'jec': 8, 'lixo': 9, 'movimentacao': 10, 
 'peticao': 11, 'procuracao': 12, 'setenca': 13}

#Prediz com a base de treimaneto texto e image

In [ ]:
y_pred = model_hybrid.predict([X_train_text, X_train_image])
y_f = []
Y_train = np.argmax(y_train, axis=1) # Convert one-hot to index
for i in range(len(y_pred)):
    pred = np.where(y_pred[i] == np.amax(y_pred[i]))
    y_f.append(pred[0][0])

print(classification_report(Y_train, y_f))

In [ ]:
X = []
lens = []
for key in X_keys[:train_size]:
    X.append([[class_mapping[i]] for i in list(docs[key]['sent_text'])])
    lens.append(len(X[-1]))
X_train = np.concatenate(X)  


In [ ]:
## OS EXPERIMENTOS ACABAM AQUI, EMBAIXO APENAS TESTES

In [ ]:
import numpy as np
from hmmlearn import hmm
import json
import pandas as pd
import numpy as np
from hmmlearn.hmm import MultinomialHMM

h = MultinomialHMM(n_components=12)
h.fit(X_train, lens)


In [ ]:
X_hmm_pp_test = []
y_hmm_test = []
y_test_true_hmm = []
for i in X_keys[train_size:]:
    X_image = docs[i]['image_path'].apply(get_array)
    X_text = docs[i]['text']
    y = to_one_hot(docs[i]['sent_text'], class_mapping, num_classes)
    
    X_test_image = image_to_numpy(X_image)
    test_data = Data(data_source=X_text,
                         alphabet=data_config["data"]["alphabet"],
                         input_size=data_config["data"]["input_size"],
                         num_of_classes=data_config["data"]["num_of_classes"],
                         alphabet_size=data_config["data"]["alphabet_size"])
    test_data.load_data()


    X_test_text = test_data.get_data_3dim()

    
    X_hmm_test = model_hybrid.predict([X_test_text, X_test_image])
    
    results_to_int  = np.argmax(X_hmm_test, axis=1)
    y_hmm = h.decode(results_to_int.reshape(-1,1))
    y_hmm_test.extend(y_hmm[1])
    y_int = docs[i]['sent_text'].map(class_mapping)
    y_test_true_hmm.extend(y_int)


In [ ]:
print(classification_report(y_test_true_hmm, y_hmm_test))

In [ ]:
num_classes = df['sent_text'].nunique()
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['sent_text']))}
print(class_mapping)
for i in range(len(docs['j3'])):
    print(class_mapping[docs['j3']['sent_text'].iloc[i]], c[1][i])


In [ ]:
from attention_decoder import AttentionDecoder

In [ ]:

def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
    return output_X, y

def get_lstm_model_with_attention(num_classes, timesteps):
    model = Sequential()
    model.add(LSTM(100, input_shape=(timesteps, num_classes), return_sequences=True))
    model.add(LSTM(100, input_shape=(timesteps, num_classes), return_sequences=True))
    model.add(LSTM(100, input_shape=(timesteps, num_classes), return_sequences=True))
    model.add(AttentionDecoder(100, num_classes))
    model.compile(loss='mse', optimizer='adam', metrics=['acc'])
    return model
def get_lstm_model_simple_reconstructor(num_classes, timesteps):
    model = Sequential()
    model.add(LSTM(128, activation='relu', input_shape=(timesteps,num_classes), return_sequences=True))
    model.add(LSTM(64, activation='relu', return_sequences=True))
    model.add(LSTM(64, activation='relu', return_sequences=True))
    model.add(LSTM(128, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(num_classes)))
    model.compile(optimizer='adam', loss='mse', metrics=['acc'])
    model.summary()
    return model

def get_lstm_model_autoencoder(num_classes, timesteps):
    model = Sequential()
    model.add(LSTM(128, activation='relu', input_shape=(timesteps,num_classes), return_sequences=True))
    model.add(LSTM(64, activation='relu', return_sequences=False))
    model.add(RepeatVector(timesteps))
    model.add(LSTM(64, activation='relu', return_sequences=True))
    model.add(LSTM(128, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(num_classes)))
    model.compile(optimizer='adam', loss='mse', metrics=['acc'])
    model.summary()
    return model

def treinar_lstm_ac(X,y, timesteps, num_classes, model_lstm):
    
    X_train, y_train = temporalize(X, y, timesteps)
    history = model_lstm.fit(X_train, y_train, epochs=100, batch_size=4)
    
    return history

def treinar_lstm_ac_2(X,y, timesteps, num_classes, model_lstm):
    #TimeSeries to Supervised
    X_train = []
    y_train = []
    print(y.shape)
    print(X.shape)
    for i in range(timesteps, len(X)):
            X_train.append(X[i-timesteps : i])
            y_train.append(y[i-timesteps : i])
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    #X_train = np.stack(X_train, axis=0)
        
    #Dimensão, número de linhas, número de colunas(timesteps) e número de features
    print(X_train.shape)
    print(y_train.shape)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], y_train.shape[2]))
    #regressor.add(LSTM(units = 50, input_shape = (X_train.shape[1], X_train.shape[2])))
   
    #treinar
    #validation_split = 0.3
    history = model_lstm.fit(X_train, y_train, epochs=50, batch_size=4)
    
    return history

def get_lstm_model(num_classes, timesteps):
    input_layer = Input(shape=(timesteps, num_classes), name='input', dtype='float32')
    #x = Conv1D(filters=256, kernel_size=4, activation='relu', input_shape=(timesteps,num_classes))(input_layer)
    #x = Dropout(0.5)(x)
    #x = MaxPooling1D(pool_size=3)(x)
    #x = Flatten()(x)
    #x = Dense(100, activation='relu')(x)
    x = layers.Bidirectional(layers.LSTM(75, input_shape = (timesteps, num_classes), return_sequences=True),name='bilstm')(input_layer)
    x = layers.Bidirectional(layers.LSTM(50, input_shape = (timesteps, num_classes)),name='bilstm2')(x)
    #x = layers.LSTM(120, input_shape = (timesteps, num_classes))(input_layer)
    #x = Dense(12, activation='relu')(x)
    #Criar Output Layer
    output = Dense(num_classes, activation='softmax', name='output')(x)
    
    model_lstm = Model(input_layer, output)
    #compilar
    model_lstm.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-4),
                    metrics=['acc'])
    
    model_lstm.summary()
    return model_lstm

def data_to_timesteps(X):
    X_train = []
    for i in range(timesteps, len(X)):
            X_train.append(X[i-timesteps : i])
            
    X_train = np.array(X_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    return X_train

def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

def treinar_lstm(X,y, timesteps, num_classes, model_lstm):
    #TimeSeries to Supervised
    X_train = []
    y_train = y
    
    for i in range(timesteps, len(X)):
            X_train.append(X[i-timesteps : i])
            
    X_train = np.array(X_train)
    
    #X_train = np.stack(X_train, axis=0)
        
    #Dimensão, número de linhas, número de colunas(timesteps) e número de features
    print(X_train.shape)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

    #regressor.add(LSTM(units = 50, input_shape = (X_train.shape[1], X_train.shape[2])))
   
    #treinar
    #validation_split = 0.3
    history = model_lstm.fit(X_train, y_train, epochs=150, batch_size=4)
    
    return history

def to_one_hot_ts(y, class_mapping, num_classes, timesteps):
    y_int = y.map(class_mapping)
    y_int = y_int[timesteps:]
    
    return to_categorical(y_int, num_classes)

def decode_timeseries(y_pred):
    y_decoded = []
    cont = 1
    len_y_true = len(y_pred) + timesteps - 1
    for i in range(len(y_pred) + timesteps - 1):
        if i < len(y_pred):
            y_decoded.append(np.argmax(y_pred[i][0]))
        else:
            y_decoded.append(np.argmax(y_pred[-1][cont]))
            cont += 1
            
    return y_decoded


In [ ]:
#bad results
timesteps = 5
model_lstm = get_lstm_model(num_classes, timesteps)
y_train_lf = []
X_lf_pp_train = []
y_true_train_docs = []
for i in X_keys[:train_size]:
    docs[i]['sent_text_t1'] = docs[i]['sent_text'].shift(-1)
    docs[i].dropna(inplace = True)
    X_image = docs[i]['image_path'].apply(get_array)
    X_text = docs[i]['text']
    
    y = to_one_hot_ts(docs[i]['sent_text_t1'], class_mapping, num_classes, timesteps)
    X_train_image = image_to_numpy(X_image)
    training_data = Data(data_source=X_text,
                         alphabet=data_config["data"]["alphabet"],
                         input_size=data_config["data"]["input_size"],
                         num_of_classes=data_config["data"]["num_of_classes"],
                         alphabet_size=data_config["data"]["alphabet_size"])
    training_data.load_data()


    X_train_text = training_data.get_data_3dim()

    
    X_lf = model_hybrid.predict([X_train_text, X_train_image])
    X_lf_pp = np.eye(X_lf.shape[1])[X_lf.argmax(1)]
    
    y_true_train_docs.append(y)
    X_lf_pp_train.append(data_to_timesteps(X_lf_pp))


    history = treinar_lstm(X_lf_pp, y, timesteps, num_classes, model_lstm)


In [ ]:
y_true_train = []
y_pred_train = []
for i in range(len(y_true_train_docs)):
    y_true_train.extend(y_true_train_docs[i])
    y_pred = model_lstm.predict(X_lf_pp_train[i])
    y_pred_train.extend(y_pred)

print(np.array(y_true_train).shape)
print(np.array(y_pred_train).shape)   
print(one_hot_decode(y_true_train))
print(one_hot_decode(y_pred_train))
print(classification_report(one_hot_decode(y_true_train), one_hot_decode(y_pred_train)))

In [ ]:
X_lf_pp_test = []
y_true_test_docs = []
for i in X_keys[train_size:]:
    docs[i]['sent_text_t1'] = docs[i]['sent_text'].shift(-1)
    docs[i].dropna(inplace = True)
    X_image = docs[i]['image_path'].apply(get_array)
    X_text = docs[i]['text']
    y = to_one_hot_ts(docs[i]['sent_text_t1'], class_mapping, num_classes, timesteps)
    
    X_test_image = image_to_numpy(X_image)
    test_data = Data(data_source=X_text,
                         alphabet=data_config["data"]["alphabet"],
                         input_size=data_config["data"]["input_size"],
                         num_of_classes=data_config["data"]["num_of_classes"],
                         alphabet_size=data_config["data"]["alphabet_size"])
    test_data.load_data()

    X_test_text = test_data.get_data_3dim()
    X_lf_test = model_hybrid.predict([X_test_text, X_test_image])
    X_lf_test_pp = np.eye(X_lf_test.shape[1])[X_lf_test.argmax(1)]
    
    y_true_test_docs.append(y)
    X_lf_pp_test.append(data_to_timesteps(X_lf_test_pp))
    


In [ ]:



y_true_test = []
y_pred_test = []
for i in range(len(y_true_test_docs)):
    y_true_test.extend(y_true_test_docs[i])
    y_pred = model_lstm.predict(X_lf_pp_test[i])
    y_pred_test.extend(y_pred)

print(np.array(y_true_test).shape)
print(np.array(y_pred_test).shape)                   
print(classification_report(one_hot_decode(y_true_test), one_hot_decode(y_pred_test)))

## SEQ TO SEQ

In [ ]:
timesteps = 3
model_lstm = get_lstm_model_with_attention(num_classes, timesteps)
y_train_lf = []
X_lf_pp_train = []
y_true_train_docs = []
for i in X_keys[:train_size]:
    X_image = docs[i]['image_path'].apply(get_array)
    X_text = docs[i]['text']
    y = to_one_hot_ts(docs[i]['sent_text'], class_mapping, num_classes, timesteps)
    
    X_train_image = image_to_numpy(X_image)
    training_data = Data(data_source=X_text,
                         alphabet=data_config["data"]["alphabet"],
                         input_size=data_config["data"]["input_size"],
                         num_of_classes=data_config["data"]["num_of_classes"],
                         alphabet_size=data_config["data"]["alphabet_size"])
    training_data.load_data()


    X_train_text = training_data.get_data_3dim()

    
    X_lf = model_hybrid.predict([X_train_text, X_train_image])
    X_lf_pp = np.eye(X_lf.shape[1])[X_lf.argmax(1)]
    
    y_true_train_docs.append(data_to_timesteps(y))
    X_lf_pp_train.append(data_to_timesteps(X_lf_pp))
    history = treinar_lstm_ac_2(X_lf_pp, y, timesteps, num_classes, model_lstm)


In [ ]:
y_true_train = []
y_pred_train = []
for i in range(len(y_true_train_docs)):
    y_true_train.extend(decode_timeseries(y_true_train_docs[i]))
    y_pred = model_lstm.predict(X_lf_pp_train[i])
    y_pred_train.extend(decode_timeseries(y_pred))

print(y_true_train)
print(y_pred_train)                   
print(classification_report(y_true_train, y_pred_train))

In [ ]:
X_lf_pp_test = []
y_true_test_docs = []
for i in X_keys[train_size:]:
    X_image = docs[i]['image_path'].apply(get_array)
    X_text = docs[i]['text']
    y = to_one_hot_ts(docs[i]['sent_text'], class_mapping, num_classes, timesteps)
    
    X_test_image = image_to_numpy(X_image)
    test_data = Data(data_source=X_text,
                         alphabet=data_config["data"]["alphabet"],
                         input_size=data_config["data"]["input_size"],
                         num_of_classes=data_config["data"]["num_of_classes"],
                         alphabet_size=data_config["data"]["alphabet_size"])
    test_data.load_data()

    X_test_text = test_data.get_data_3dim()
    X_lf_test = model_hybrid.predict([X_test_text, X_test_image])
    X_lf_test_pp = np.eye(X_lf_test.shape[1])[X_lf_test.argmax(1)]
    
    y_true_test_docs.append(data_to_timesteps(y))
    X_lf_pp_test.append(data_to_timesteps(X_lf_test_pp))
    


In [ ]:
y_true_test = []
y_pred_test = []
for i in range(len(y_true_test_docs)):
    y_true_test.extend(decode_timeseries(y_true_test_docs[i]))
    y_pred = model_lstm.predict(X_lf_pp_test[i])
    y_pred_test.extend(decode_timeseries(y_pred))

print(y_true_test)
print(y_pred_test)                   
print(classification_report(y_true_test, y_pred_test))

In [ ]:
print(X_train_text.shape)
print(X_test_text.shape)
print(X_train_image.shape)
print(X_test_image.shape)
print(y_train.shape)
print(y_test.shape)

X_text = np.concatenate((X_train_text, X_test_text))
X_image = np.concatenate((X_train_image, X_test_image))
y = np.concatenate((y_train, y_test))

print(X_text.shape)
print(X_image.shape)
print(y.shape)

In [ ]:
X_image = df['image_path'].apply(get_array)
X_text = df['text']
y = to_one_hot(df['sent_text'], class_mapping, num_classes)

In [ ]:
text_data = Data(data_source=X_text,
                         alphabet=data_config["data"]["alphabet"],
                         input_size=data_config["data"]["input_size"],
                         num_of_classes=data_config["data"]["num_of_classes"],
                         alphabet_size=data_config["data"]["alphabet_size"])
text_data.load_data()
X_text_array = text_data.get_data_3dim()


In [ ]:
def image_to_numpy(image_df):
    all_images = []
    for i in range(len(image_df)):
        all_images.append(image_df.iloc[i])  
    return np.array(all_images)

X_image_array = image_to_numpy(X_image)

In [ ]:
print(y_test[0])

In [ ]:
from tensorflow.keras.models import load_model
model_hybrid = load_model('model_ness_txt_image_v1.h5')



In [ ]:
layer_name = 'concatanated'
  
layer_output = model_hybrid.get_layer(layer_name).output

intermediate_model = Model(inputs=model_hybrid.input,outputs=layer_output)

encoded_page = intermediate_model.predict([X_text_array, X_image_array])


In [ ]:
print(type(encoded_page))
print(encoded_page.shape)

In [ ]:
data = {}
data['encoded_page'] = []
for i in encoded_page:
    data['encoded_page'].append(i)
    
df_encoded = pd.DataFrame.from_dict(data)
df = pd.concat([df, df_encoded], axis=1, sort=False)
display(df.head())

In [ ]:
X_encoded = df['encoded_page']
y = to_one_hot(df['sent_text'])
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.25, random_state=42)

X_train = np.stack(X_train, axis=0)
X_test = np.stack(X_test, axis=0)

In [ ]:
encoded_input =Input(shape=(512,), name='encoded_page', dtype='float32')
#x = Dense(1024, activation='relu')(encoded_page)
output = Dense(num_classes, activation='softmax', name='output')(encoded_input)
model = Model(encoded_input, output)
model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-4),
                    metrics=['acc'])

In [ ]:
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
#X = df[['encoded_page','image_page', 'image_name', 'sent_text']]
X = df[['encoded_page', 'image_name', 'sent_text']]
#X['sent_text_one_hot'] = to_one_hot(X['sent_text'])
docs = {}

values = X['image_name'].value_counts()
print(values.index)
print(values['12'])

for i in values.index:
    docs[i] = X[X['image_name'] == i].sort_values(by=['image_name']).drop(['image_name'], axis=1)
    

In [ ]:
def treinar_lstm(X,y, timesteps):
    #TimeSeries to Supervised
    X_train = []
    y_train = y
    
    for i in range(timesteps, len(X)):
            X_train.append(X[i-timesteps : i])
            
    X_train = np.array(X_train)
    
    #X_train = np.stack(X_train, axis=0)
        
    #Dimensão, número de linhas, número de colunas(timesteps) e número de features
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    #print(X_train.shape)
    #print(y_train.shape)

    #regressor.add(LSTM(units = 50, input_shape = (X_train.shape[1], X_train.shape[2])))
    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]), name='input', dtype='float32')
    x = layers.Bidirectional(layers.LSTM(512, input_shape = (X_train.shape[1], X_train.shape[2])), name='bilstm')(input_layer)
    #x = layers.LSTM(50, input_shape = (X_train.shape[1], X_train.shape[2]))(input_layer)
    #Criar Output Layer
    output = Dense(8, activation='softmax', name='output')(x)
    
    model_lstm = Model(input_layer, output)
    #compilar
    model_lstm.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-4),
                    metrics=['acc'])
    
    model_lstm.summary()
    #treinar
    history = model_lstm.fit(X_train, y_train, epochs=150, batch_size=32, validation_split = 0.3)
    
    return model_lstm, history

In [ ]:
def to_one_hot_ts(y, timesteps):
    num_classes = y.nunique()
    class_mapping = {label:idx for idx,label in enumerate(np.unique(y))}
    y_int = y.map(class_mapping)
    y_int = y_int[timesteps:]
    
    return to_categorical(y_int, num_classes)


In [ ]:
#df_test = docs['12'].drop(['sent_text', 'image_page], axis=1)
df_test = docs['12'].drop(['sent_text'], axis=1)

X_test = df_test['encoded_page']

y_test = to_one_hot(docs['12']['sent_text'])
timesteps = 5

regressor, history = treinar_lstm(X_test,y_test, timesteps)

In [ ]:
X_test = df_test['encoded_page'][5:]
y_test = to_one_hot(docs['12']['sent_text'])

X_test = np.stack(X_test, axis=0)

encoded_input =Input(shape=(512,), name='encoded_page', dtype='float32')
#x = Dense(1024, activation='relu')(encoded_page)
output = Dense(8, activation='softmax', name='output')(encoded_input)
model = Model(encoded_input, output)
model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-4),
                    metrics=['acc'])
model.fit(X_test, y_test, epochs=1000, batch_size=32, validation_split = 0.3)